# Explore Brains During the Menstrual Cycle

This notebook loads and visualizes the brain data available in the folder `/home/data/28andme`

## Set-up

In [ ]:
import src.setcwd

src.setcwd.main()

## Imports

In [ ]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import nibabel
import numpy as np
from IPython.display import HTML
from nilearn import plotting
from PIL import Image, ImageDraw, ImageFont

import src.viz as viz

## Visualize Brain Images from Anatomical T1w

In [ ]:
img = nibabel.load("/home/data/28andme/sub-01/ses-02/anat/sub-01_ses-02_T1w.nii.gz")
print(img.header)

In [ ]:
img_data = img.get_fdata()
print(img_data.shape)

slice_0 = img_data[26, :, :]
slice_1 = img_data[:, 30, :]
slice_2 = img_data[:, :, 16]
_ = viz.show_slices([slice_0, slice_1, slice_2])

In [ ]:
view = plotting.view_img(img, bg_img=False)
view

## Visualize Brain Images from Anatomical T2hipp

In [ ]:
img = nibabel.load("/home/data/28andme/sub-01/ses-02/anat/sub-01_ses-02_T2hipp.nii.gz")
print(img.header)
img_data = img.get_fdata()
print(img_data.shape)

slice_0 = img_data[26, :, :]
slice_1 = img_data[:, 30, :]
slice_2 = img_data[:, :, 16]
_ = viz.show_slices([slice_0, slice_1, slice_2])

In [ ]:
view = plotting.view_img(img, bg_img=False)
view

## Visualize Images from the Segmentation ASHS left_lfseg_corr

In [ ]:
img = nibabel.load(
    "/home/data/28andme/sub-01/ses-02/ashs/left_lfseg_corr_usegray_CT_LQ.nii.gz"
)
img_data = img.get_fdata()
print(len(np.unique(img_data.flatten())))
# Set value 0 to "bad data"
img_data = np.ma.masked_where(img_data < 0.05, img_data)
print(img_data.shape)

cmap = mpl.cm.get_cmap("tab20b").copy()
cmaplist = [cmap(2 * i) for i in range(10)]
cmap = mpl.colors.LinearSegmentedColormap.from_list("Custom cmap", cmaplist, 10)
cmap.set_bad(color="black")

fig, axes = plt.subplots(nrows=1, ncols=2)
im = axes[0].imshow(img_data[120:200, 120:200, 16], cmap=cmap)
fig.colorbar(im, cax=axes[-1]);

### Inspect Segmentation: 9 classes

In [ ]:
img_vec = img_data.flatten()
img_mask = img_vec[img_vec != 0.0]
plt.hist(img_mask)

In [ ]:
view = plotting.view_img(img, bg_img=False, cmap=cmap)
view

## Visualize Images from the Segmentation ASHS right_lfseg_corr

In [ ]:
img = nibabel.load(
    "/home/data/28andme/sub-01/ses-02/ashs/right_lfseg_corr_usegray_CT_LQ.nii.gz"
)
img_data = img.get_fdata()
print(img.header)
print(img_data.shape)

slice_0 = img_data[26, :, :]
slice_1 = img_data[:, 30, :]
slice_2 = img_data[220:320, 120:220, 16]
_ = viz.show_slices([slice_0, slice_1, slice_2])

In [ ]:
view = plotting.view_img(img, bg_img=False, cmap=cmap)
view

## Play Video of 2D Slice from a Time-Series of 3D Images

Choose which video to play, among:
- anat/sub-01_ses-**_T1w.nii.gz
- anat/sub-01_ses-**_T2hipp.nii.gz
- ashs/left_lfseg_corr_usegray_CT_LQ.nii.gz
- ashs/right_lfseg_corr_usegray_CT_LQ.nii.gz

In [ ]:
anima = viz.animate("ashs/left_lfseg_corr_usegray_CT_LQ.nii.gz", img_dir="/home/data/28andme/", slice_z=16)
HTML(anima.to_html5_video())

In [ ]:
anima = viz.animate("ashs/right_lfseg_corr_usegray_CT_LQ.nii.gz", slice_z=16)
HTML(anima.to_html5_video())

# Make video of Laura's brain for presentation

Creating paths to brain data, one path for each day, contained in nii_paths.

Note that you need to choose which data_dir to use and only use half of the code in the below cell.

In [ ]:
# Choose either this data_dir
data_dir = "/home/data/28andme/sub-01"
nii_paths = []
for day in range(2, 31):
    nii_paths.append(f"{data_dir}/ses-{day:02}/anat/sub-01_ses-{day:02}_T1w.nii.gz")
print(nii_paths)

print("\n")
# Or choose this data_dir
# data_dir = "/home/data/28andMeOC_correct"
# hemisphere = "left"
# day_dirs = [os.path.join(data_dir, f"Day{i:02d}") for i in range(1, 61)]

# print(f"Looking into: {day_dirs}")
# nii_paths = []
# for i_day, day_dir in enumerate(day_dirs):
#     for file_name in os.listdir(day_dir):
#         if file_name.startswith(hemisphere) and file_name.endswith(".nii.gz"):
#             nii_paths.append(os.path.join(day_dir, file_name))
#             break
# print(nii_paths)

In [ ]:
# Set the directory to save the converted images
anim_dir = os.path.join(os.getcwd(), "my28brains/results/anims")
tmp_dir = os.path.join(os.getcwd(), "my28brains/results/tmp")

print(f"Will save the animation in {anim_dir} and the temporary images in {tmp_dir}.")

In [ ]:
for i_path, path in enumerate(nii_paths):
    img = nibabel.load(path).get_fdata()
    img = (img / img.max() * 255).astype("uint8")  # Normalize pixel values
    image_path = os.path.join(tmp_dir, f"mri_day_{i_path:03}.png")
    # brain shifts, need to take different slices for later days.
    if i_path < 7 and i_path != 5:
        Image.fromarray(img[:, 50, :]).save(image_path)
    else:
        Image.fromarray(img[:, 50, :]).save(image_path)

In [ ]:
output_file = os.path.join(anim_dir, "mris.mp4")
frame_rate = 3

# Use ffmpeg to create the movie from the images
os.system(
    f"ffmpeg -framerate {frame_rate} -pattern_type glob -i '{tmp_dir}/mri_day_*.png' -c:v libx264 -pix_fmt yuv420p {output_file}"
)

In [ ]:
font_size = 50

# Set the font style
font = ImageFont.truetype(
    os.path.join(anim_dir, "DejaVuSerif.ttf"),
    font_size,
)


for day in range(2, 31):
    # Set the text to display
    text = f"Day {day:02}"

    # Get the width and height of the text
    text_width, text_height = font.getsize(text)

    # Create a new image with a white background
    image = Image.new("RGB", (text_width, text_height), color=(255, 255, 255))

    # Add the text to the image
    draw = ImageDraw.Draw(image)
    draw.text((0, 0), text, fill=(0, 0, 0), font=font)

    # Save the image as a .png file
    path = os.path.join(tmp_dir, f"Day_{day:02}.png")
    image.save(path, "PNG")

In [ ]:
# Set the output file name and path
output_file = os.path.join(anim_dir, "mris_days.mp4")
frame_rate = 3

# Use ffmpeg to create the movie from the images
os.system(
    f"ffmpeg -framerate {frame_rate} -pattern_type glob -i '{tmp_dir}/Day_*.png' -c:v libx264 -pix_fmt yuv420p {output_file}"
)